## 加载数据 Dataset

- Dataset : 提供一种方式去获取数据及其label
    - 如何获取每一个数据及其label
    - 总共有多少数据
  
- Dataloader： 为后面的网络提供不同的数据形式

In [17]:
from torch.utils.data import Dataset
from PIL import Image
import os

In [5]:
help(Dataset)

Help on class Dataset in module torch.utils.data.dataset:

class Dataset(typing.Generic)
 |  Dataset(*args, **kwds)
 |  
 |  An abstract class representing a :class:`Dataset`.
 |  
 |  All datasets that represent a map from keys to data samples should subclass
 |  it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
 |  data sample for a given key. Subclasses could also optionally overwrite
 |  :meth:`__len__`, which is expected to return the size of the dataset by many
 |  :class:`~torch.utils.data.Sampler` implementations and the default options
 |  of :class:`~torch.utils.data.DataLoader`.
 |  
 |  .. note::
 |    :class:`~torch.utils.data.DataLoader` by default constructs a index
 |    sampler that yields integral indices.  To make it work with a map-style
 |    dataset with non-integral indices/keys, a custom sampler must be provided.
 |  
 |  Method resolution order:
 |      Dataset
 |      typing.Generic
 |      builtins.object
 |  
 |  Methods defined 

In [19]:
class MyData(Dataset):
    def __init__(self,root_dir,label_dir):
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path = os.path.join(self.root_dir,self.label_dir)
        self.img_path = os.listdir(self.path)
    def __getitem__(self,idx):
        img_name = self.img_path[idx]
        img_item_path = os.path.join(self.root_dir,self.label_dir,img_name)
        img = Image.open(img_item_path)
        label = self.label_dir
        return img,label
    def __len__(self):
        return len(self.img_path)
        

In [28]:
root_dir = "./hymenoptera_data/hymenoptera_data/train"
ants_label_dir = "ants"
bees_label_dir = "bees"
ants_dataset = MyData(root_dir,ants_label_dir)
# img,label = ants_dataset[0]
# img.show()
bees_dataset = MyData(root_dir,bees_label_dir)
train_dataset = ants_dataset + bees_dataset
len(train_dataset),len(ants_dataset),len(bees_dataset)
# img,label = train_dataset[125]
# img.show()

In [10]:
# from PIL import Image
# img_path = "./hymenoptera_data/hymenoptera_data/train/ants/0013035.jpg"
# img = Image.open(img_path)
# img.show()

In [13]:
# dir_path = "./hymenoptera_data/hymenoptera_data/train/ants/"
# import os
# img_path_list = os.listdir(dir_path)
# img_path_list
# root_dir =  "./hymenoptera_data/hymenoptera_data/train"
# label_dir = "ants"
# os.path.join(root_dir,label_dir)

'./hymenoptera_data/hymenoptera_data/train\\ants'

## Tensorboard的使用

pip install tensorboard

### add_scalar

In [40]:
from torch.utils.tensorboard import  SummaryWriter
writer = SummaryWriter("logs")

# writer.add_image()

##y=x
for i in range(100):

    writer.add_scalar("y=2x",2*i,i)

writer.close()

In [43]:
!tensorboard --logdir=logs --port=6007

^C


tensorboard参数： --logdir=文件夹名称  --port=端口号  可以不设置端口号就是默认，为了避免冲突可以设置

由于多次添加，导致log文件中有各种，可能导致tensorboard显示曲线的一些拟合什么的。

解决：
- 删除logs文件夹下的文件，重新运行程序。
- 对应每个任务每次都设立子文件夹。

### add_image
数据类型需要tensor或者numpy array

In [49]:
from torch.utils.tensorboard import  SummaryWriter
from PIL import Image
import numpy as np

writer = SummaryWriter("logs")
img_path = "./hymenoptera_data/hymenoptera_data/train/ants/0013035.jpg"
img_PIL = Image.open(img_path)
img_array = np.array(img_PIL)
print(img_array.shape)
print(type(img_array))
writer.add_image("test",img_array,1,dataformats="HWC")##默认是CHW,如果不是需要进行设置

for i in range(100):
    writer.add_scalar("y=2x",3*i,i)
writer.close()

(512, 768, 3)
<class 'numpy.ndarray'>


In [52]:
!tensorboard --logdir=logs --port=6008

^C


### 如何打开事件文件

In [ ]:
!tensorboard --logdir=logs --port=6008

## Transform的使用
包含 totensor,resize等，一个进行图形转换的工具箱

### transform.ToTensor
- transform如何使用
    1. 创建具体工具： tensor_trans = transform.ToTensor()
    2. 使用工具： result = tensor_trans(img)
- 为什么要使用tensor数据类型

In [53]:
from torchvision import transforms
from PIL import Image

In [54]:
img_path = "./hymenoptera_data/hymenoptera_data/train/ants/0013035.jpg"
img = Image.open(img_path)
print(img)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x512 at 0x18452F18EC8>


In [57]:
tensor_trans = transforms.ToTensor()##可以将PIL或者numpy数组类型的图片转换成tensor
tensor_img = tensor_trans(img)
# tensor_img

In [59]:
writer = SummaryWriter("logs")

writer.add_image("img test",tensor_img)

In [60]:
!tensorboard --logdir=logs --port=6008


^C


In [61]:
import cv2 ##opencv读取图片
cv_img=cv2.imread(img_path)

### 常见的transform


不同图片格式：
- PIL Image.open()
- tensor ToTensor()
- numpy arrays cv.imread()

In [76]:
from PIL import Image
from torchvision import transforms
writer = SummaryWriter("logs")
img_path = "./hymenoptera_data/hymenoptera_data/train/ants/0013035.jpg"
img = Image.open(img_path)
tensor_trans = transforms.ToTensor()##可以将PIL或者numpy数组类型的图片转换成tensor
tensor_img = tensor_trans(img)
print(tensor_img[0][0][0])

tensor(0.3137)


transform.Compose 将几个transform方法进行组合

- transform.ToTensor()
- transform.ToPILImage()

#### transform.Nomalize() 归一化

input[channel] = (input[channel] - mean[channel])/std[channel]


In [77]:
trans_norm = transforms.Normalize([1,3,5],[3,2,1])
img_norm = trans_norm(tensor_img)
print(img_norm[0][0][0])
writer.add_image("Normalize",img_norm,1)
writer.close()

tensor(-0.2288)


In [122]:
!tensorboard --logdir=logs --port=6008

^C


#### transform.Resize
类似于等比缩放 

输入是PILImage

In [87]:
print(img.size) ##原来是768*512
trans_resize= transforms.Resize((512,512))
img_resize = trans_resize(img)###被变成512*512
print(img_resize) 
trans_totensor = transforms.ToTensor()
img_resize = trans_totensor(img_resize)
writer.add_image("Resize",img_resize,0)

(768, 512)
<PIL.Image.Image image mode=RGB size=512x512 at 0x1845141A788>


#### transform.Compose

In [100]:
trans_resize_2 = transforms.Resize(512)##将长宽中较长的那个设成512.另一个等比例放大
trans_compose = transforms.Compose([trans_resize_2,trans_totensor]) ##注意有先后顺序
img_resize_2 = trans_compose(img)
print(img_resize_2.shape)
print(img.size)
writer.add_image("Resize",img_resize_2,1)

torch.Size([3, 512, 768])
(768, 512)


#### RandomCrop
随机裁剪，传入裁剪值。传入一个值，就会裁剪成正方形。

In [104]:
trans_random = transforms.RandomCrop(512)
trans_compose_2 = transforms.Compose([trans_random,trans_totensor])
for i in range(10):
    img_crop = trans_compose_2(img)
    writer.add_image("RandomCrop",img_crop,i)


In [109]:
trans_random = transforms.RandomCrop((500,100))
trans_compose_2 = transforms.Compose([trans_random,trans_totensor])
for i in range(10):
    img_crop = trans_compose_2(img)
    writer.add_image("RandomCrop_hw",img_crop,i)


## torchvision中的数据集使用

In [112]:
import torchvision
torchvision.__version__

'0.8.2'

看官网api

https://pytorch.org/vision/0.8/datasets.html#cifar

In [113]:
train_set = torchvision.datasets.CIFAR10(root="./dataset",train=True,download=True)
test_set = torchvision.datasets.CIFAR10(root="./dataset",train=False,download=True)


Extracting ./dataset\cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


In [119]:
print(test_set[0])
print(test_set.classes)
img,target = test_set[0]
print(img)
print(target)
print(test_set.classes[target])
img.show()

(<PIL.Image.Image image mode=RGB size=32x32 at 0x184522CD108>, 3)
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
<PIL.Image.Image image mode=RGB size=32x32 at 0x184522CD648>
3
cat


下载数据集并进行transform

In [127]:
import torchvision
dataset_transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor()]
)

train_set = torchvision.datasets.CIFAR10(root="./dataset/",train=True,transform=dataset_transform,download=True)
test_set = torchvision.datasets.CIFAR10(root="./dataset/",train=False,transform=dataset_transform,download=True)
print(test_set[0])

Files already downloaded and verified
Files already downloaded and verified
(tensor([[[0.6196, 0.6235, 0.6471,  ..., 0.5373, 0.4941, 0.4549],
         [0.5961, 0.5922, 0.6235,  ..., 0.5333, 0.4902, 0.4667],
         [0.5922, 0.5922, 0.6196,  ..., 0.5451, 0.5098, 0.4706],
         ...,
         [0.2667, 0.1647, 0.1216,  ..., 0.1490, 0.0510, 0.1569],
         [0.2392, 0.1922, 0.1373,  ..., 0.1020, 0.1137, 0.0784],
         [0.2118, 0.2196, 0.1765,  ..., 0.0941, 0.1333, 0.0824]],

        [[0.4392, 0.4353, 0.4549,  ..., 0.3725, 0.3569, 0.3333],
         [0.4392, 0.4314, 0.4471,  ..., 0.3725, 0.3569, 0.3451],
         [0.4314, 0.4275, 0.4353,  ..., 0.3843, 0.3725, 0.3490],
         ...,
         [0.4863, 0.3922, 0.3451,  ..., 0.3804, 0.2510, 0.3333],
         [0.4549, 0.4000, 0.3333,  ..., 0.3216, 0.3216, 0.2510],
         [0.4196, 0.4118, 0.3490,  ..., 0.3020, 0.3294, 0.2627]],

        [[0.1922, 0.1843, 0.2000,  ..., 0.1412, 0.1412, 0.1294],
         [0.2000, 0.1569, 0.1765,  ..., 0.1216

In [128]:
writer = SummaryWriter("p10")
for i in range(10):
    img,target = test_set[i]
    writer.add_image("test_set_1",img,i)
writer.close()


In [124]:
!tensorboard --logdir=p10 --port=6009

^C


## DataLoader的使用

https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader

dataset (Dataset) – dataset from which to load the data.

batch_size (int, optional) – how many samples per batch to load (default: 1).

shuffle (bool, optional) – set to True to have the data reshuffled at every epoch (default: False).

sampler (Sampler or Iterable, optional) – defines the strategy to draw samples from the dataset. Can be any Iterable with __len__ implemented. If specified, shuffle must not be specified.

batch_sampler (Sampler or Iterable, optional) – like sampler, but returns a batch of indices at a time. Mutually exclusive with batch_size, shuffle, sampler, and drop_last.

num_workers (int, optional) – how many subprocesses to use for data loading. 0 means that the data will be loaded in the main process. (default: 0)

collate_fn (callable, optional) – merges a list of samples to form a mini-batch of Tensor(s). Used when using batched loading from a map-style dataset.

pin_memory (bool, optional) – If True, the data loader will copy Tensors into device/CUDA pinned memory before returning them. If your data elements are a custom type, or your collate_fn returns a batch that is a custom type, see the example below.

drop_last (bool, optional) – set to True to drop the last incomplete batch, if the dataset size is not divisible by the batch size. If False and the size of dataset is not divisible by the batch size, then the last batch will be smaller. (default: False)

In [138]:
import torchvision
from torch.utils.data import DataLoader
test_data = torchvision.datasets.CIFAR10(root="./dataset/",train=False,transform=torchvision.transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_data,batch_size=4,shuffle=True,num_workers=0,drop_last=False)
img,target = test_data[0]
# img.shape
# target

for data in  test_loader:
    imgs,targets = data
#     print(imgs.shape)
#     print(targets)

Files already downloaded and verified
torch.Size([4, 3, 32, 32])
tensor([2, 4, 2, 7])
torch.Size([4, 3, 32, 32])
tensor([8, 1, 9, 4])
torch.Size([4, 3, 32, 32])
tensor([0, 0, 3, 7])
torch.Size([4, 3, 32, 32])
tensor([9, 2, 9, 7])
torch.Size([4, 3, 32, 32])
tensor([7, 4, 6, 4])
torch.Size([4, 3, 32, 32])
tensor([9, 1, 3, 0])
torch.Size([4, 3, 32, 32])
tensor([3, 2, 8, 5])
torch.Size([4, 3, 32, 32])
tensor([9, 2, 6, 5])
torch.Size([4, 3, 32, 32])
tensor([0, 6, 0, 1])
torch.Size([4, 3, 32, 32])
tensor([2, 0, 1, 8])
torch.Size([4, 3, 32, 32])
tensor([2, 7, 6, 7])
torch.Size([4, 3, 32, 32])
tensor([3, 0, 0, 6])
torch.Size([4, 3, 32, 32])
tensor([3, 4, 6, 8])
torch.Size([4, 3, 32, 32])
tensor([2, 6, 8, 6])
torch.Size([4, 3, 32, 32])
tensor([7, 2, 2, 8])
torch.Size([4, 3, 32, 32])
tensor([1, 8, 3, 7])
torch.Size([4, 3, 32, 32])
tensor([3, 9, 7, 2])
torch.Size([4, 3, 32, 32])
tensor([1, 2, 7, 2])
torch.Size([4, 3, 32, 32])
tensor([2, 1, 5, 2])
torch.Size([4, 3, 32, 32])
tensor([3, 0, 2, 5])
to

tensor([4, 4, 7, 3])
torch.Size([4, 3, 32, 32])
tensor([7, 7, 5, 9])
torch.Size([4, 3, 32, 32])
tensor([3, 3, 2, 9])
torch.Size([4, 3, 32, 32])
tensor([3, 0, 0, 4])
torch.Size([4, 3, 32, 32])
tensor([7, 3, 1, 3])
torch.Size([4, 3, 32, 32])
tensor([5, 1, 7, 4])
torch.Size([4, 3, 32, 32])
tensor([3, 5, 8, 2])
torch.Size([4, 3, 32, 32])
tensor([2, 4, 9, 2])
torch.Size([4, 3, 32, 32])
tensor([5, 2, 5, 4])
torch.Size([4, 3, 32, 32])
tensor([5, 6, 8, 6])
torch.Size([4, 3, 32, 32])
tensor([5, 1, 9, 7])
torch.Size([4, 3, 32, 32])
tensor([6, 4, 7, 7])
torch.Size([4, 3, 32, 32])
tensor([6, 3, 8, 4])
torch.Size([4, 3, 32, 32])
tensor([8, 2, 5, 6])
torch.Size([4, 3, 32, 32])
tensor([8, 5, 1, 2])
torch.Size([4, 3, 32, 32])
tensor([7, 6, 1, 3])
torch.Size([4, 3, 32, 32])
tensor([7, 4, 3, 5])
torch.Size([4, 3, 32, 32])
tensor([7, 0, 9, 3])
torch.Size([4, 3, 32, 32])
tensor([0, 4, 8, 0])
torch.Size([4, 3, 32, 32])
tensor([0, 8, 7, 2])
torch.Size([4, 3, 32, 32])
tensor([5, 3, 8, 9])
torch.Size([4, 3, 3

torch.Size([4, 3, 32, 32])
tensor([9, 4, 3, 6])
torch.Size([4, 3, 32, 32])
tensor([3, 4, 3, 9])
torch.Size([4, 3, 32, 32])
tensor([6, 5, 3, 2])
torch.Size([4, 3, 32, 32])
tensor([7, 3, 2, 3])
torch.Size([4, 3, 32, 32])
tensor([5, 3, 4, 6])
torch.Size([4, 3, 32, 32])
tensor([0, 2, 6, 6])
torch.Size([4, 3, 32, 32])
tensor([8, 3, 6, 2])
torch.Size([4, 3, 32, 32])
tensor([7, 3, 6, 9])
torch.Size([4, 3, 32, 32])
tensor([6, 1, 8, 9])
torch.Size([4, 3, 32, 32])
tensor([9, 1, 6, 0])
torch.Size([4, 3, 32, 32])
tensor([5, 9, 9, 1])
torch.Size([4, 3, 32, 32])
tensor([2, 1, 6, 8])
torch.Size([4, 3, 32, 32])
tensor([4, 8, 1, 3])
torch.Size([4, 3, 32, 32])
tensor([8, 5, 2, 5])
torch.Size([4, 3, 32, 32])
tensor([3, 2, 9, 2])
torch.Size([4, 3, 32, 32])
tensor([9, 0, 8, 4])
torch.Size([4, 3, 32, 32])
tensor([2, 7, 6, 6])
torch.Size([4, 3, 32, 32])
tensor([5, 9, 1, 6])
torch.Size([4, 3, 32, 32])
tensor([6, 6, 5, 2])
torch.Size([4, 3, 32, 32])
tensor([8, 5, 7, 5])
torch.Size([4, 3, 32, 32])
tensor([5, 5,

torch.Size([4, 3, 32, 32])
tensor([3, 8, 3, 7])
torch.Size([4, 3, 32, 32])
tensor([0, 0, 6, 2])
torch.Size([4, 3, 32, 32])
tensor([7, 6, 1, 9])
torch.Size([4, 3, 32, 32])
tensor([2, 8, 4, 0])
torch.Size([4, 3, 32, 32])
tensor([2, 2, 8, 7])
torch.Size([4, 3, 32, 32])
tensor([4, 0, 2, 1])
torch.Size([4, 3, 32, 32])
tensor([2, 6, 5, 7])
torch.Size([4, 3, 32, 32])
tensor([7, 0, 9, 7])
torch.Size([4, 3, 32, 32])
tensor([7, 9, 6, 0])
torch.Size([4, 3, 32, 32])
tensor([8, 1, 8, 7])
torch.Size([4, 3, 32, 32])
tensor([1, 7, 6, 2])
torch.Size([4, 3, 32, 32])
tensor([9, 7, 8, 7])
torch.Size([4, 3, 32, 32])
tensor([4, 0, 1, 6])
torch.Size([4, 3, 32, 32])
tensor([9, 8, 2, 4])
torch.Size([4, 3, 32, 32])
tensor([6, 2, 4, 4])
torch.Size([4, 3, 32, 32])
tensor([0, 8, 7, 9])
torch.Size([4, 3, 32, 32])
tensor([4, 9, 2, 1])
torch.Size([4, 3, 32, 32])
tensor([6, 0, 6, 9])
torch.Size([4, 3, 32, 32])
tensor([8, 7, 8, 2])
torch.Size([4, 3, 32, 32])
tensor([3, 5, 0, 6])
torch.Size([4, 3, 32, 32])
tensor([9, 2,

torch.Size([4, 3, 32, 32])
tensor([3, 9, 4, 8])
torch.Size([4, 3, 32, 32])
tensor([7, 5, 6, 1])
torch.Size([4, 3, 32, 32])
tensor([5, 5, 4, 3])
torch.Size([4, 3, 32, 32])
tensor([3, 4, 2, 4])
torch.Size([4, 3, 32, 32])
tensor([1, 5, 2, 9])
torch.Size([4, 3, 32, 32])
tensor([5, 9, 2, 6])
torch.Size([4, 3, 32, 32])
tensor([0, 7, 6, 8])
torch.Size([4, 3, 32, 32])
tensor([4, 7, 7, 1])
torch.Size([4, 3, 32, 32])
tensor([4, 4, 0, 2])
torch.Size([4, 3, 32, 32])
tensor([3, 6, 8, 4])
torch.Size([4, 3, 32, 32])
tensor([7, 4, 3, 8])
torch.Size([4, 3, 32, 32])
tensor([6, 4, 2, 4])
torch.Size([4, 3, 32, 32])
tensor([6, 3, 8, 5])
torch.Size([4, 3, 32, 32])
tensor([6, 5, 9, 9])
torch.Size([4, 3, 32, 32])
tensor([6, 4, 2, 7])
torch.Size([4, 3, 32, 32])
tensor([3, 3, 6, 9])
torch.Size([4, 3, 32, 32])
tensor([5, 1, 3, 8])
torch.Size([4, 3, 32, 32])
tensor([4, 1, 3, 3])
torch.Size([4, 3, 32, 32])
tensor([1, 9, 7, 8])
torch.Size([4, 3, 32, 32])
tensor([6, 2, 4, 6])
torch.Size([4, 3, 32, 32])
tensor([5, 5,

...

torch.Size([4, 3, 32, 32])
tensor([6, 1, 5, 8])
torch.Size([4, 3, 32, 32])
tensor([1, 9, 9, 1])
torch.Size([4, 3, 32, 32])
tensor([3, 4, 1, 2])
torch.Size([4, 3, 32, 32])
tensor([2, 6, 5, 1])
torch.Size([4, 3, 32, 32])
tensor([0, 2, 3, 1])
torch.Size([4, 3, 32, 32])
tensor([7, 6, 6, 1])
torch.Size([4, 3, 32, 32])
tensor([6, 8, 6, 4])
torch.Size([4, 3, 32, 32])
tensor([9, 0, 2, 3])
torch.Size([4, 3, 32, 32])
tensor([5, 3, 4, 7])
torch.Size([4, 3, 32, 32])
tensor([5, 3, 1, 6])
torch.Size([4, 3, 32, 32])
tensor([8, 9, 1, 4])
torch.Size([4, 3, 32, 32])
tensor([7, 0, 7, 2])
torch.Size([4, 3, 32, 32])
tensor([4, 3, 7, 9])
torch.Size([4, 3, 32, 32])
tensor([1, 7, 2, 0])
torch.Size([4, 3, 32, 32])
tensor([2, 1, 9, 9])
torch.Size([4, 3, 32, 32])
tensor([7, 1, 8, 2])
torch.Size([4, 3, 32, 32])
tensor([8, 3, 2, 8])
torch.Size([4, 3, 32, 32])
tensor([7, 3, 8, 9])
torch.Size([4, 3, 32, 32])
tensor([5, 8, 2, 8])
torch.Size([4, 3, 32, 32])
tensor([9, 6, 3, 8])
torch.Size([4, 3, 32, 32])
tensor([6, 3,

tensor([5, 1, 8, 6])
torch.Size([4, 3, 32, 32])
tensor([6, 3, 5, 6])
torch.Size([4, 3, 32, 32])
tensor([8, 3, 7, 0])
torch.Size([4, 3, 32, 32])
tensor([9, 6, 3, 8])
torch.Size([4, 3, 32, 32])
tensor([4, 5, 0, 9])
torch.Size([4, 3, 32, 32])
tensor([0, 7, 5, 6])
torch.Size([4, 3, 32, 32])
tensor([1, 1, 6, 7])
torch.Size([4, 3, 32, 32])
tensor([1, 7, 2, 3])
torch.Size([4, 3, 32, 32])
tensor([1, 0, 4, 3])
torch.Size([4, 3, 32, 32])
tensor([5, 7, 1, 1])
torch.Size([4, 3, 32, 32])
tensor([4, 3, 1, 8])
torch.Size([4, 3, 32, 32])
tensor([4, 0, 7, 1])
torch.Size([4, 3, 32, 32])
tensor([5, 5, 3, 6])
torch.Size([4, 3, 32, 32])
tensor([8, 4, 9, 0])
torch.Size([4, 3, 32, 32])
tensor([0, 2, 9, 0])
torch.Size([4, 3, 32, 32])
tensor([6, 6, 0, 7])
torch.Size([4, 3, 32, 32])
tensor([0, 7, 0, 3])
torch.Size([4, 3, 32, 32])
tensor([5, 2, 8, 9])
torch.Size([4, 3, 32, 32])
tensor([0, 4, 8, 5])
torch.Size([4, 3, 32, 32])
tensor([9, 6, 9, 0])
torch.Size([4, 3, 32, 32])
tensor([8, 5, 8, 2])
torch.Size([4, 3, 3

torch.Size([4, 3, 32, 32])
tensor([0, 4, 6, 3])
torch.Size([4, 3, 32, 32])
tensor([4, 5, 7, 5])
torch.Size([4, 3, 32, 32])
tensor([8, 9, 4, 5])
torch.Size([4, 3, 32, 32])
tensor([6, 5, 7, 4])
torch.Size([4, 3, 32, 32])
tensor([4, 9, 6, 0])
torch.Size([4, 3, 32, 32])
tensor([9, 6, 8, 6])
torch.Size([4, 3, 32, 32])
tensor([0, 9, 5, 3])
torch.Size([4, 3, 32, 32])
tensor([0, 9, 3, 7])
torch.Size([4, 3, 32, 32])
tensor([0, 8, 2, 9])
torch.Size([4, 3, 32, 32])
tensor([8, 7, 5, 8])
torch.Size([4, 3, 32, 32])
tensor([9, 1, 0, 6])
torch.Size([4, 3, 32, 32])
tensor([9, 8, 1, 4])
torch.Size([4, 3, 32, 32])
tensor([6, 5, 7, 0])
torch.Size([4, 3, 32, 32])
tensor([5, 5, 5, 2])
torch.Size([4, 3, 32, 32])
tensor([2, 2, 5, 6])
torch.Size([4, 3, 32, 32])
tensor([8, 7, 3, 8])
torch.Size([4, 3, 32, 32])
tensor([8, 1, 0, 7])
torch.Size([4, 3, 32, 32])
tensor([9, 2, 5, 6])
torch.Size([4, 3, 32, 32])
tensor([0, 0, 5, 0])
torch.Size([4, 3, 32, 32])
tensor([4, 9, 6, 8])
torch.Size([4, 3, 32, 32])
tensor([3, 4,

tensor([5, 4, 6, 8])
torch.Size([4, 3, 32, 32])
tensor([2, 4, 3, 5])
torch.Size([4, 3, 32, 32])
tensor([1, 6, 6, 9])
torch.Size([4, 3, 32, 32])
tensor([2, 7, 7, 6])
torch.Size([4, 3, 32, 32])
tensor([7, 5, 7, 6])
torch.Size([4, 3, 32, 32])
tensor([9, 5, 5, 4])
torch.Size([4, 3, 32, 32])
tensor([7, 5, 1, 6])
torch.Size([4, 3, 32, 32])
tensor([7, 0, 3, 7])
torch.Size([4, 3, 32, 32])
tensor([9, 3, 0, 6])
torch.Size([4, 3, 32, 32])
tensor([3, 8, 1, 4])
torch.Size([4, 3, 32, 32])
tensor([9, 9, 6, 3])
torch.Size([4, 3, 32, 32])
tensor([8, 6, 6, 7])
torch.Size([4, 3, 32, 32])
tensor([2, 8, 5, 5])
torch.Size([4, 3, 32, 32])
tensor([6, 2, 4, 1])
torch.Size([4, 3, 32, 32])
tensor([9, 6, 2, 4])
torch.Size([4, 3, 32, 32])
tensor([9, 7, 1, 6])
torch.Size([4, 3, 32, 32])
tensor([9, 2, 3, 1])
torch.Size([4, 3, 32, 32])
tensor([1, 0, 0, 9])
torch.Size([4, 3, 32, 32])
tensor([7, 0, 9, 6])
torch.Size([4, 3, 32, 32])
tensor([0, 3, 9, 6])
torch.Size([4, 3, 32, 32])
tensor([6, 2, 1, 3])
torch.Size([4, 3, 3

tensor([4, 1, 4, 5])
torch.Size([4, 3, 32, 32])
tensor([0, 8, 4, 8])
torch.Size([4, 3, 32, 32])
tensor([2, 3, 5, 2])
torch.Size([4, 3, 32, 32])
tensor([8, 7, 3, 7])
torch.Size([4, 3, 32, 32])
tensor([6, 1, 7, 1])
torch.Size([4, 3, 32, 32])
tensor([4, 8, 9, 1])
torch.Size([4, 3, 32, 32])
tensor([1, 8, 2, 7])
torch.Size([4, 3, 32, 32])
tensor([1, 7, 8, 9])
torch.Size([4, 3, 32, 32])
tensor([2, 6, 2, 5])
torch.Size([4, 3, 32, 32])
tensor([0, 1, 7, 3])
torch.Size([4, 3, 32, 32])
tensor([4, 1, 2, 5])
torch.Size([4, 3, 32, 32])
tensor([8, 4, 9, 2])
torch.Size([4, 3, 32, 32])
tensor([7, 4, 0, 5])
torch.Size([4, 3, 32, 32])
tensor([2, 1, 9, 6])
torch.Size([4, 3, 32, 32])
tensor([0, 2, 5, 5])
torch.Size([4, 3, 32, 32])
tensor([2, 1, 3, 8])
torch.Size([4, 3, 32, 32])
tensor([8, 0, 0, 7])
torch.Size([4, 3, 32, 32])
tensor([5, 0, 5, 0])
torch.Size([4, 3, 32, 32])
tensor([2, 8, 8, 3])
torch.Size([4, 3, 32, 32])
tensor([5, 0, 8, 3])
torch.Size([4, 3, 32, 32])
tensor([4, 4, 0, 0])
torch.Size([4, 3, 3

使用DataLoader之后，图片会被批量处理。返回的test_loader[0]实际上包含batch_size张图片。

In [139]:
import torchvision
from torch.utils.data import DataLoader
from torch.utils.tensorboard import  SummaryWriter
test_data = torchvision.datasets.CIFAR10(root="./dataset/",train=False,transform=torchvision.transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_data,batch_size=64,shuffle=True,num_workers=0,drop_last=False)##最后剩余的不被batchsize整除的保留
img,target = test_data[0]
# img.shape
# target
writer = SummaryWriter("dataloader")
step = 0
for data in  test_loader:
    imgs,targets = data
#     print(imgs.shape)
#     print(targets)
    writer.add_images("test_data",imgs,step) ##这里使用add_images进行批量的显示
    step+=1
writer.close()

Files already downloaded and verified


In [143]:
!tensorboard --logdir=dataloader

^C
